In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from pprint import pprint
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2

2023-11-30 16:53:35.488153: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 16:53:35.524942: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 16:53:35.524985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 16:53:35.526001: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 16:53:35.536095: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 16:53:35.536787: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode

HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions

FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions


PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions

In [4]:
hand_model_path = 'models/hand_landmarkr_full.task'
face_model_path = 'models/face_landmarker.task'
pose_model_path = 'models/pose_landmarker_heavy.task'

In [5]:
handOptions = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=hand_model_path),
    running_mode=VisionRunningMode.VIDEO,
    num_hands=2)

faceOptions = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=face_model_path),
    running_mode=VisionRunningMode.VIDEO)

poseOptions = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=pose_model_path),
    running_mode=VisionRunningMode.VIDEO)

In [22]:
def initModels():
    handLandmarker=HandLandmarker.create_from_options(handOptions)
    faceLandmarker=FaceLandmarker.create_from_options(faceOptions)
    poseLandmarker=PoseLandmarker.create_from_options(poseOptions)
    return handLandmarker,faceLandmarker,poseLandmarker
handLandmarker=HandLandmarker.create_from_options(handOptions)
faceLandmarker=FaceLandmarker.create_from_options(faceOptions)
poseLandmarker=PoseLandmarker.create_from_options(poseOptions)
# initModels()

I0000 00:00:1701343691.044574   32434 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701343691.045782   33907 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
I0000 00:00:1701343691.078906   32434 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701343691.079824   33926 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
W0000 00:00:1701343691.080097   32434 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
I0000 00:00:1701343691.102990   32434 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701343691.104057   33945 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5

In [7]:
def draw_landmarks_on_image(rgb_image, pose_detection_result,hand_detection_result,face_detection_results):
  pose_landmarks_list = pose_detection_result.pose_landmarks
  
  hand_landmarks_list = hand_detection_result.hand_landmarks
  handedness_list = hand_detection_result.handedness
  
  face_landmarks_list = face_detection_results.face_landmarks

  annotated_image = np.copy(rgb_image)

  MARGIN = 10  # pixels
  FONT_SIZE = 1
  FONT_THICKNESS = 1
  HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
        annotated_image,
        pose_landmarks_proto,
        solutions.pose.POSE_CONNECTIONS,
        solutions.drawing_styles.get_default_pose_landmarks_style())
    
  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
        annotated_image,
        hand_landmarks_proto,
        solutions.hands.HAND_CONNECTIONS,
        solutions.drawing_styles.get_default_hand_landmarks_style(),
        solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.
    height, width, _ = annotated_image.shape
    x_coordinates = [landmark.x for landmark in hand_landmarks]
    y_coordinates = [landmark.y for landmark in hand_landmarks]
    text_x = int(min(x_coordinates) * width)
    text_y = int(min(y_coordinates) * height) - MARGIN

    # Draw handedness (left or right hand) on the image.
    cv2.putText(annotated_image, f"{handedness[0].category_name}",
                (text_x, text_y), cv2.FONT_HERSHEY_DUPLEX,
                FONT_SIZE, HANDEDNESS_TEXT_COLOR, FONT_THICKNESS, cv2.LINE_AA)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_contours_style())
    solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_iris_connections_style())
  return annotated_image

In [8]:
def save_data(name,pose_landmarket_result, hand_landmarker_result, face_landmarket_result):
    if len(face_landmarket_result.face_landmarks) > 0:
        finalFaceData: np.ndarray = np.array([[i.x, i.y, i.z]
                                              for i in face_landmarket_result.face_landmarks[0]]).flatten()
    else:
        finalFaceData= np.zeros((1434,))
    
    if len(pose_landmarket_result.pose_landmarks) > 0:
        finalPoseData: np.ndarray = np.array([[i.x, i.y, i.z]
                                              for i in pose_landmarket_result.pose_landmarks[0]]).flatten()
    else:
        finalPoseData = np.zeros((99,))
    finalLeftHandData = np.zeros((63,))
    finalRightHandData = np.zeros((63,))
    for idx in range(len(hand_landmarker_result.hand_landmarks)):
        
        if hand_landmarker_result.handedness[idx][0].category_name == "Left":
            finalLeftHandData=np.array([[i.x, i.y, i.z] for i in hand_landmarker_result.hand_landmarks[[idx][0]]]).flatten()
        else:
            finalRightHandData = np.array([[i.x, i.y, i.z] for i in hand_landmarker_result.hand_landmarks[[idx][0]]]).flatten()


    np.save(name, np.concatenate([finalFaceData, finalPoseData, finalLeftHandData, finalRightHandData]))

In [20]:
cap = cv2.VideoCapture(0)
prev_frame_time = 0
new_frame_time = 0
startTime=time.time()
while cap.isOpened():
    new_frame_time = time.time()

    ret, frame = cap.read()

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    hand_landmarker_result = handLandmarker.detect_for_video(
        mp_image,int((time.time()-startTime)*1000))
    face_landmarket_result = faceLandmarker.detect_for_video(
        mp_image,int((time.time()-startTime)*1000))
    pose_landmarket_result = poseLandmarker.detect_for_video(
        mp_image,int((time.time()-startTime)*1000))

    image = cv2.cvtColor(draw_landmarks_on_image(
        mp_image.numpy_view(), pose_landmarket_result,hand_landmarker_result,face_landmarket_result), cv2.COLOR_RGB2BGR)

    # pprint(hand_landmarker_result.handedness)
    save_data("testSave.npy",pose_landmarket_result, hand_landmarker_result, face_landmarket_result)

    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
    cv2.putText(image, f"{fps:1f}", (15, 50),
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 3)
    cv2.imshow('OpenCV Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [9]:
cv2.destroyAllWindows()

In [10]:
sign=['control','thankyou','hello','iloveyou']

In [11]:
DATA_PATH = os.path.join('data')
actions = np.array(sign)
no_sequences = 30
sequence_length = 15

In [10]:
for action in actions:
    os.mkdir(DATA_PATH+f'/{action}')
    for i in range(no_sequences):
        os.mkdir(DATA_PATH+f'/{action}/{i}')

FileExistsError: [Errno 17] File exists: 'data/control'

In [ ]:
cap.release()
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

In [24]:
cap = cv2.VideoCapture(0)
for action in actions:
    input()
    for sequence in range(no_sequences):
        input()
        handLandmarker,faceLandmarker,poseLandmarker=initModels()
        startTime=time.time()
        for frame_num in range(sequence_length):
            ret, frame = cap.read()
            
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

            hand_landmarker_result = handLandmarker.detect_for_video(
                mp_image,int((time.time()-startTime)*1000))
            face_landmarket_result = faceLandmarker.detect_for_video(
                mp_image,int((time.time()-startTime)*1000))
            pose_landmarket_result = poseLandmarker.detect_for_video(
                mp_image,int((time.time()-startTime)*1000))

            image = cv2.cvtColor(draw_landmarks_on_image(
                mp_image.numpy_view(), pose_landmarket_result,hand_landmarker_result,face_landmarket_result), cv2.COLOR_RGB2BGR)

            if frame_num == 0: 
                cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255, 255), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {} Frame No {}'.format(action, sequence,frame_num), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(500)
            else: 
                cv2.putText(image, 'Collecting frames for {} Video Number {} Frame No {}'.format(action, sequence,frame_num), (15, 12),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                # Show to screen
                cv2.imshow('OpenCV Feed', image)
            
            npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
            save_data(npy_path, pose_landmarket_result,
                      hand_landmarker_result, face_landmarket_result)
            # Break gracefully
            cv2.imshow('OpenCV Feed', image)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
                
cap.release()
cv2.destroyAllWindows()

I0000 00:00:1701343699.915395   32434 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701343699.918752   34055 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
I0000 00:00:1701343699.942790   32434 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701343699.943755   34074 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5.12-300.fc39.x86_64)
W0000 00:00:1701343699.944005   32434 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
I0000 00:00:1701343699.972882   32434 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1701343699.974137   34093 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1), renderer: AMD Radeon Graphics (renoir, LLVM 16.0.6, DRM 3.54, 6.5